- [View Solution Notebook](./solution.html)
- [View Project Page]()

# Imports and Instantiation

In [1]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer

# 1. Instantiate DistilGPT-2's `tokenizer` and `model` using the `.from_pretrained` method.
tokenizer = GPT2Tokenizer.from_pretrained("distilgpt2")
model = GPT2LMHeadModel.from_pretrained("distilgpt2")

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

c:\Users\anush\AppData\Local\Programs\Python\Python312\Lib\site-packages\huggingface_hub\file_download.py:157: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\anush\.cache\huggingface\hub\models--distilgpt2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

c:\Users\anush\AppData\Local\Programs\Python\Python312\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/762 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/353M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

## Tokenization and Generation

In [2]:
# 2. Assign pt_tensors the input text's tokens in PyTorch tensor form
def encode_text_as_pt_tensor(text):
    pt_tensors = tokenizer.encode(text, return_tensors="pt")
    return pt_tensors

print(encode_text_as_pt_tensor("hello, world!"))

tensor([[31373,    11,   995,     0]])


In [3]:
from transformers import set_seed
set_seed(42)
# 3. Use set_seed to make the rest of the notebook's output deterministic. Pass it the number 42.
# set seed here

In [5]:
prompt = "Your prompt here!"
tokens = encode_text_as_pt_tensor(prompt)
# 4. Instruct the model to generate a completion on your choice of prompt using the greedy search method.
# Pass pad_token_id=tokenizer.eos_token_id as the second argument to prevent seeing a warning.
output_tokens = model.generate(tokens, pad_token_id = tokenizer.eos_token_id)
output_tokens

c:\Users\anush\AppData\Local\Programs\Python\Python312\Lib\site-packages\transformers\generation\utils.py:1168: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


tensor([[7120, 6152,  994,    0,  198,  198,  198,  198,  198,  198,  198,  198,
          198,  198,  198,  198,  198,  198,  198,  198]])

In [6]:
# 5. Decode the resulting tokens.
def decode_tokens(tokens):
    text = tokenizer.decode(tokens)
    return text

decode_tokens(output_tokens[0])

'Your prompt here!\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n'

## Experimenting with Generation Strategies

In [10]:
# 6. Adapt the function below to use beam search in its generations. Then call it three times with 2 beam, 6 beams, and 14 beams.
# Pass pad_token_id=tokenizer.eos_token_id to model.generate to prevent seeing a warning.
def generate_with_beam_search(prompt,num_beams):
    tokens = encode_text_as_pt_tensor(prompt)
    output = model.generate(tokens, num_beams=num_beams, pad_token_id = tokenizer.eos_token_id)
    completion = decode_tokens(output[0])
    print(completion)
    return completion


prompt = "Discipline is what you hate to do but to do it like you love it"
generate_with_beam_search(prompt,14)
# make multiple calls to the model here

Discipline is what you hate to do but to do it like you love it."





'Discipline is what you hate to do but to do it like you love it."\n\n\n'

In [16]:
# 7. Repeat the same process you did with beam search on step 6 with n-gram penalties here.
# Call our function three times with n_gram values of 2, 3, and 4.
# Pass pad_token_id=tokenizer.eos_token_id to model.generate to prevent seeing a warning.
def generate_with_ngram_penalty(prompt, n_gram_penalty, num_beams=6):
    tokens = encode_text_as_pt_tensor(prompt)
    output = model.generate(tokens,no_repeat_ngram_size = n_gram_penalty, num_beams=num_beams, pad_token_id = tokenizer.eos_token_id)
    completion = decode_tokens(output[0])
    print(completion)
    return completion

generate_with_ngram_penalty(prompt,2)
# make multiple calls to the function here

c:\Users\anush\AppData\Local\Programs\Python\Python312\Lib\site-packages\transformers\generation\utils.py:1168: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


Discipline is what you hate to do but to do it like you love it.”



'Discipline is what you hate to do but to do it like you love it.”\n'

In [18]:
# 8. Same as steps 6 and 7, experiment with different settings of temperature and top_k here after instructing the model to do sampling.
# Choose your own values for temperature and top k and see how the model's output responds.
# Pass pad_token_id=tokenizer.eos_token_id to model.generate to prevent seeing a warning.

def generate_with_sampling(prompt, temperature, top_k, n_gram_penalty=2):
    tokens = encode_text_as_pt_tensor(prompt)
    output = model.generate(tokens,no_repeat_ngram_size = n_gram_penalty,temperature=temperature, top_k=top_k,do_sample = True, pad_token_id = tokenizer.eos_token_id)
    completion = decode_tokens(output[0])
    print(f"Temperature: {temperature}\nTop K: {top_k}\n {completion}")
    return completion

# make multiple calls to the function here
generate_with_sampling(prompt,0.6,50)


c:\Users\anush\AppData\Local\Programs\Python\Python312\Lib\site-packages\transformers\generation\utils.py:1168: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


Temperature: 0.6
Top K: 50
 Discipline is what you hate to do but to do it like you love it.

This


'Discipline is what you hate to do but to do it like you love it.\n\nThis'

## Using CodeCarbon

In [20]:
! pip install codecarbon

   ---------------------------------------- 0.0/496.1 kB ? eta -:--:--
   --- ----------------------------------- 41.0/496.1 kB 991.0 kB/s eta 0:00:01
   -------------------- ------------------- 256.0/496.1 kB 3.2 MB/s eta 0:00:01
   ---------------------------------------- 496.1/496.1 kB 4.5 MB/s eta 0:00:00
   ---------------------------------------- 0.0/53.1 kB ? eta -:--:--
   ---------------------------------------- 53.1/53.1 kB 2.9 MB/s eta 0:00:00
   ---------------------------------------- 0.0/1.6 MB ? eta -:--:--
   ---------------- ----------------------- 0.7/1.6 MB 20.2 MB/s eta 0:00:01
   ---------------------------------------  1.6/1.6 MB 20.1 MB/s eta 0:00:01
   ---------------------------------------  1.6/1.6 MB 20.1 MB/s eta 0:00:01
   ---------------------------------------- 1.6/1.6 MB 11.3 MB/s eta 0:00:00



[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [21]:
from codecarbon import track_emissions

# 9. Add the CodeCarbon decorator to the line directly above this function.
# Then fill the function in with your preferred settings.

# add the decorator here
def generate_with_sampling(prompt, temperature, top_k, n_gram_penalty=2):
    tokens = encode_text_as_pt_tensor(prompt)
    output = model.generate(tokens,no_repeat_ngram_size = n_gram_penalty,temperature=temperature, top_k=top_k,do_sample = True, pad_token_id = tokenizer.eos_token_id)
    completion = decode_tokens(output[0])
    print(f"Temperature: {temperature}\nTop K: {top_k}\n {completion}")
    return completion

generate_with_sampling("Carbon dioxide is a", 0.6, 50)

c:\Users\anush\AppData\Local\Programs\Python\Python312\Lib\site-packages\transformers\generation\utils.py:1168: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


Temperature: 0.6
Top K: 50
 Carbon dioxide is a potent greenhouse gas. Researchers at the University of Texas at Austin found that,


'Carbon dioxide is a potent greenhouse gas. Researchers at the University of Texas at Austin found that,'

In [ ]:
# import pandas as pd
# # 10. Use pandas' read_csv method to load in the emissions.csv we generated.
# # Then, print the first few rows with emissions.head().

# df = pd.read_csv()

In [ ]:
# import matplotlib.pyplot as plt

# plt.figure(figsize=(10, 5))
# # 11. Pass the `emissions` DataFrame's 'duration' column to the x-axis (first argument) and the 'emissions' column to the y-axis (second argument).
# plt.scatter # pass the columns here, then include `color='blue', alpha=0.6)`
# plt.title('Emissions by Duration')
# plt.xlabel('Duration (seconds)')
# plt.ylabel('Emissions (kg CO2)')
# plt.grid(True)
# plt.show()